In [1]:
import random
import cnn_builder as cbuild
import config
import copy
import glob
import importlib
import lipiodol_methods as lm
import niftiutils.helper_fxns as hf
import niftiutils.transforms as tr
import niftiutils.masks as masks
import numpy as np
import os
from os.path import *
import shutil
from scipy.ndimage.morphology import binary_closing, binary_opening, binary_dilation
from skimage.morphology import ball, label

from keras import backend as K
from keras.engine import Input, Model
from keras.layers import *
from keras.optimizers import Adam
from keras.utils import np_utils
%matplotlib inline

C:\Users\Clinton\AppData\Local\conda\conda\envs\old-keras\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
importlib.reload(config)
C = config.Config()

In [3]:
img_dir = "D:\\Lipiodol\\Images all"
seg_dir = "D:\\Lipiodol\\Images extracted and segmentations"
target_dir = "D:\\Lipiodol\\Data"

In [92]:
seg = glob.glob(join(target_dir,"*","wholeliver_24hCT*.ids"))[0]

In [ ]:
lesion_id = "07"
glob.glob(join(target_dir,lesion_id,"CT24h","*"))

In [27]:
for f in glob.glob(join(target_dir,lesion_id,"CT24h","*")):
    if "5 0" not in f:
        shutil.rmtree(f)
    else:
        shutil.move()

# MRI

### Data prep

In [4]:
# MRI
for seg in glob.glob(join(target_dir,"*","masks","liver bl.ids")):
    lesion_id = dirname(seg)[len(target_dir)+1:-len("masks")-1]
    if exists(join(C.full_img_dir, lesion_id+"_mri_art.npy")):
        continue
    
    mri_img, mri_dims = hf.nii_load(join(target_dir, lesion_id, "MRI-BL", "t1-art.nii.gz"))
    mask, _ = masks.get_mask(seg, mri_dims, mri_img.shape)

    mri_img -= mri_img.min()
    mri_img /= mri_img.std()
    
    np.save(join(C.full_img_dir, lesion_id+"_mri_art.npy"), mri_img)
    np.save(join(C.full_img_dir, lesion_id+"_mr_bl_liver_mask.npy"), mask > 128)
    
    #mri_img, mri_dims = hf.nii_load(join(target_dir, lesion_id, "MRI-BL", "t1-pre.nii.gz"), True, True)
    #mri_img -= mri_img.min()
    #mri_img /= mri_img.std()
    #np.save(join(C.full_img_dir, lesion_id+"_mri_art.npy"), mri_img)

In [13]:
data_dir = "D:\\JHU\\JHU HCC"

for path in glob.glob(join(data_dir,"*","*.nii")):
    lesion_id = basename(dirname(path))
    lesion_id = lesion_id[:lesion_id.find(" ")]
    
    mri_img, mri_dims = hf.nii_load(path)
    
    mask_paths = glob.glob(join(dirname(path),"*Segs","Liver","*.ids"))
    if len(mask_paths) != 1:
        print(path)
        continue
    mask, _ = masks.get_mask(mask_paths[0], mri_dims, mri_img.shape)

    mri_img -= mri_img.min()
    mri_img /= mri_img.std()
    
    np.save(join(C.full_img_dir, lesion_id+"_mri_art.npy"), mri_img)
    np.save(join(C.full_img_dir, lesion_id+"_mr_bl_liver_mask.npy"), mask > 128)

### Train model

In [ ]:
importlib.reload(cbuild)
model = cbuild.build_unet()
#model.summary(120)

In [ ]:
hist = model.fit_generator(cbuild.train_gen_mri(), epochs=500, steps_per_epoch=5)

In [20]:
model.save(join(C.model_dir, "mri_liver.hdf5"))

### Test

In [72]:
importlib.reload(lm)

<module 'lipiodol_methods' from 'C:\\Users\\Clinton\\Documents\\Lipiodol\\lipiodol_methods.py'>

In [81]:
lesion_id = "12"
mri_nii_path = join(target_dir, lesion_id, "MRI-30d", "t1-art.nii.gz")
save_path = join(target_dir, lesion_id, "masks", "mri_30d_liver")
tumor_mask_path = join(target_dir, lesion_id, "masks", "tumor_30dFU_MRI")
lm.seg_liver_mri_from_path(mri_nii_path, save_path, model, tumor_mask_path)

# CT

In [13]:
for fn in glob.glob(join(C.full_img_dir, "*")):
    if fn.endswith("_ct.npy") and (not fn.startswith("PK") and not fn.startswith("BM")):
        os.remove(fn)
        os.remove(fn[:-6]+"mask.npy")

In [21]:
for seg in glob.glob(join(target_dir,"*","masks","wholeliver_24hCT*.ids")):
    lesion_id = dirname(seg)[len(target_dir)+1:-len("masks")-1]
    if exists(join(C.full_img_dir, lesion_id+"_ct.npy")):
        continue
    
    ct_img, ct_dims = hf.nii_load(join(target_dir, lesion_id, "nii_files", "ct24.nii.gz"))
    mask, _ = masks.get_mask(seg, ct_dims, ct_img.shape)

    ct_img = tr.apply_window(ct_img)
    ct_img -= np.amin(ct_img)
    
    np.save(join(C.full_img_dir, lesion_id+"_ct.npy"), ct_img)
    np.save(join(C.full_img_dir, lesion_id+"_mask.npy"), mask > mask.max()/2)

In [4]:
importlib.reload(cbuild)
model = cbuild.build_unet()
model.summary(120)

________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
input_1 (InputLayer)                   (None, 128, 128, 64)       0                                                     
________________________________________________________________________________________________________________________
reshape_1 (Reshape)                    (None, 128, 128, 64, 1)    0             input_1[0][0]                           
________________________________________________________________________________________________________________________
conv3d_1 (Conv3D)                      (None, 128, 128, 64, 8)    152           reshape_1[0][0]                         
________________________________________________________________________________________________________________________
batch_normalization_1 (BatchNorm

In [5]:
hist = model.fit_generator(cbuild.train_gen_ct(), epochs=500, steps_per_epoch=5)

Epoch 1/500
5/5 [==============================] - 14s 3s/step - loss: 0.8301 - acc: 0.4766
Epoch 2/500
5/5 [==============================] - 8s 2s/step - loss: 0.7098 - acc: 0.5776
Epoch 3/500
5/5 [==============================] - 12s 2s/step - loss: 0.6228 - acc: 0.7290
Epoch 4/500
5/5 [==============================] - 6s 1s/step - loss: 0.5823 - acc: 0.8249
Epoch 5/500
5/5 [==============================] - 9s 2s/step - loss: 0.5090 - acc: 0.8928
Epoch 6/500
5/5 [==============================] - 7s 1s/step - loss: 0.5349 - acc: 0.9024
Epoch 7/500
5/5 [==============================] - 7s 1s/step - loss: 0.4863 - acc: 0.9113
Epoch 8/500
5/5 [==============================] - 6s 1s/step - loss: 0.5096 - acc: 0.9172
Epoch 9/500
5/5 [==============================] - 8s 2s/step - loss: 0.4495 - acc: 0.9422
Epoch 10/500
5/5 [==============================] - 12s 2s/step - loss: 0.4165 - acc: 0.9591
Epoch 11/500
5/5 [==============================] - 9s 2s/step - loss: 0.4058 - acc: 0

Epoch 177/500
5/5 [==============================] - 8s 2s/step - loss: 0.0306 - acc: 0.9894
Epoch 178/500
5/5 [==============================] - 5s 1s/step - loss: 0.0264 - acc: 0.9909
Epoch 179/500
5/5 [==============================] - 8s 2s/step - loss: 0.0271 - acc: 0.9912
Epoch 180/500
5/5 [==============================] - 10s 2s/step - loss: 0.0279 - acc: 0.9900
Epoch 181/500
5/5 [==============================] - 8s 2s/step - loss: 0.0287 - acc: 0.9896
Epoch 182/500
5/5 [==============================] - 8s 2s/step - loss: 0.0255 - acc: 0.9920
Epoch 183/500
5/5 [==============================] - 7s 1s/step - loss: 0.0296 - acc: 0.9893
Epoch 184/500
5/5 [==============================] - 6s 1s/step - loss: 0.0247 - acc: 0.9924
Epoch 185/500
5/5 [==============================] - 8s 2s/step - loss: 0.0266 - acc: 0.9904
Epoch 186/500
5/5 [==============================] - 6s 1s/step - loss: 0.0270 - acc: 0.9912
Epoch 187/500
5/5 [==============================] - 5s 987ms/step - 

5/5 [==============================] - 5s 1s/step - loss: 0.0203 - acc: 0.9916
Epoch 352/500
5/5 [==============================] - 5s 943ms/step - loss: 0.0180 - acc: 0.9927
Epoch 353/500
5/5 [==============================] - 4s 879ms/step - loss: 0.0183 - acc: 0.9927
Epoch 354/500
5/5 [==============================] - 6s 1s/step - loss: 0.0188 - acc: 0.9923
Epoch 355/500
5/5 [==============================] - 5s 952ms/step - loss: 0.0214 - acc: 0.9910
Epoch 356/500
5/5 [==============================] - 5s 1s/step - loss: 0.0190 - acc: 0.9922
Epoch 357/500
5/5 [==============================] - 6s 1s/step - loss: 0.0190 - acc: 0.9925
Epoch 358/500
5/5 [==============================] - 5s 1s/step - loss: 0.0213 - acc: 0.9915
Epoch 359/500
5/5 [==============================] - 5s 934ms/step - loss: 0.0192 - acc: 0.9922
Epoch 360/500
5/5 [==============================] - 8s 2s/step - loss: 0.0195 - acc: 0.9921
Epoch 361/500
5/5 [==============================] - 6s 1s/step - loss: 

In [6]:
model.save(join(C.model_dir, "ct_liver.hdf5"))

In [107]:
lesion_id = "18"
ct_img, ct_dims = hf.dcm_load(join(target_dir, lesion_id, "CT24h"), True, True)
lm.seg_liver_ct(ct_img, join(target_dir, lesion_id, "ct24_liver"), ct_dims, model)

# Misc

In [12]:
info = {}

for seg in glob.glob(join(target_dir,"*","wholeliver_24hCT*.ids")):
    if dirname(seg) in info.keys():
        (ct_img, ct_dims, mask) = info[dirname(seg)]
    else:
        ct_img, ct_dims = hf.dcm_load(join(dirname(seg), "CT24h"), True, True)
        
    seg_liver(ct_img, dirname(seg), ct_dims)
    info[dirname(seg)] = (ct_img, ct_dims)
    break